# 20240920
# MODULE/01.4 ★ load_nau_EQPID_추가_240920  를 그대로 카피함. 

# 20240922 : UNIQUE_ID에 M_TIME 구분자 추가
 현재 unique_id : m_time 구분자 없음. 
 문제점 : 동일wafer가 adi, oco 데이터 둘다 존재하면 에러발생.  (M_STEPSEQ 정보가 없음...ㅜ )
 변경점 : unique_id에 m_time 추가함. 


 # 20240922 : M_STEP 정보추가
 현재 : M_STEP 정보가 NAU파일 내부에 없음.
 문제점 : ADI, OCO 섞여있으면 스텝구분을 못해줌..
 변경점 : 파일제목에 있는 M_STEP정보를 끌고 오게 변경 

 

 #########################################################################################################

 # 20241003 
 copy from  "01.5 Load nau 정리_240922.ipynb"
 Oncell residual로부터 Ideal PSM 을 뽑기 위해 제작
 
 1. PSM 전처리 과정 추가 (nau파일로부터 'PerShorMRC"시트에서 정보 끌어옴)



In [4]:
import pandas as pd
import numpy as np
import os
from openpyxl import load_workbook
import openpyxl
import time
from datetime import datetime


# nau 파일이 있는 폴더 경로
folder_path = 'C:/py_data/nau'


################################################ NAU파일제목에서 M_STEP정보 끌어와서 시트에 저장  ########################################

# 폴더 내의 모든 파일에 대해 실행
for file_name in os.listdir(folder_path):
    # 파일 경로 설정
    file_path = os.path.join(folder_path, file_name)
    
    # 파일인지 확인 (폴더는 제외)
    if os.path.isfile(file_path) and file_name.endswith('.nau'):
        try:
            # 파일 확장자를 임시로 .xlsx로 변경
            temp_file_path = file_path.replace('.nau', '.xlsx')
            os.rename(file_path, temp_file_path)

            # pandas를 이용해 엑셀 파일(유사)을 읽기
            excel_data = pd.read_excel(temp_file_path, engine='openpyxl')
            print(f"{file_name} 파일 읽기 성공")

            # 파일명을 "_" 기준으로 분할
            file_name_without_extension = os.path.splitext(file_name)[0]
            split_file_name = file_name_without_extension.split("_")

            # 기존 엑셀 파일 열기
            workbook = load_workbook(temp_file_path)

            # 파일명 시트를 첫 번째로 추가
            new_sheet_name = "FileName"
            if new_sheet_name not in workbook.sheetnames:
                new_sheet = workbook.create_sheet(title=new_sheet_name, index=0)  # 첫 번째 시트로 추가
            else:
                new_sheet = workbook[new_sheet_name]

            # 분할된 파일명을 첫 번째 시트에 기록
            for col, value in enumerate(split_file_name, start=1):
                new_sheet.cell(row=1, column=col, value=value)

            # 변경된 파일 저장
            workbook.save(temp_file_path)
            print(f"{file_name} 파일에 새로운 시트 추가 완료")

            # 확장자를 다시 .nau로 복원
            os.rename(temp_file_path, file_path)

        except Exception as e:
            print(f"{file_name} 파일을 읽는 중 에러 발생: {e}")




################################################ NAU로부터 필요데이터 추출 (전처리) ########################################




# 추출할 컬럼 위치 설정 (예: 첫 번째 열은 0, 두 번째 열은 1로 인덱스 시작)
columns_to_extract = [0, 1, 2, 3, 4, 5, 6, 7]  # Wafer, TEST, DieX, DieY, X_reg, Y_reg, MRC_X, MRC_Y의 열 위치

def process_nau_files(folder_path, columns_to_extract):
    # 결과를 담을 리스트 생성
    combined_rawdata_list = []
    combined_trocs_input_list = []
    mrc_data_list = []
    combined_psm_input_list = []

    # 폴더 내 모든 nau 파일에 대해 반복
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.nau'):
            file_path = os.path.join(folder_path, file_name)
            
            # 필요한 시트만 읽기
            rawdata_file = pd.read_excel(file_path, sheet_name='RawData-1')
            trocs_input_file = pd.read_excel(file_path, sheet_name='Trocs Input')
            psm_input_file = pd.read_excel(file_path, sheet_name='PerShotMRC')

            ########### m_step 시트 불러오기 #########################
            m_step_file = pd.read_excel(file_path, sheet_name='FileName')

            ######################################## RawData-1 전처리 ##################################################
            # 지정된 열 추출
            extracted_data_raw = rawdata_file.iloc[:, columns_to_extract].copy()

            # 'LOT_ID' 값 및 'STEPSEQ' 추출
            lot_id_value_raw = rawdata_file.columns[13]
            stepseq_value_raw = rawdata_file.iloc[0, 13]
            wafer_value_raw = rawdata_file.iloc[0, 0]
            p_eqpid__value_raw = rawdata_file.iloc[1, 13]
            photo_ppid_value_raw = rawdata_file.iloc[11, 13]
            p_time_value_raw = rawdata_file.iloc[2, 13]
            m_time_value_raw = rawdata_file.iloc[4, 13]
            chuckid_value_raw = rawdata_file.iloc[15, 13]

            # 'm_step' 값 추출 및 신규컬럼 추가
            m_step_value_raw = m_step_file.columns[1]
            extracted_data_raw['M_STEP'] = m_step_value_raw

                
            # 새로운 컬럼 추가
            extracted_data_raw['STEPSEQ'] = stepseq_value_raw
            extracted_data_raw['LOT_ID'] = lot_id_value_raw

            # 추가 정보 추출 및 컬럼 추가
            extracted_data_raw['STEP_PITCH_X'] = rawdata_file.iloc[6, 13]
            extracted_data_raw['STEP_PITCH_Y'] = rawdata_file.iloc[7, 13]
            extracted_data_raw['MAP_SHIFT_X'] = rawdata_file.iloc[8, 13]
            extracted_data_raw['MAP_SHIFT_Y'] = rawdata_file.iloc[9, 13]

            # 'coordinate_X', 'coordinate_Y' 매핑
            coord_map = rawdata_file[['Test No', 'coordinate_X', 'coordinate_Y']].drop_duplicates(subset='Test No').set_index('Test No')
            extracted_data_raw['coordinate_X'] = extracted_data_raw['TEST'].map(coord_map['coordinate_X'])
            extracted_data_raw['coordinate_Y'] = extracted_data_raw['TEST'].map(coord_map['coordinate_Y'])
                  
            # 'wf_x' 및 'wf_y' 계산
            extracted_data_raw['wf_x'] = (
                extracted_data_raw['DieX'] * extracted_data_raw['STEP_PITCH_X'] +
                extracted_data_raw['MAP_SHIFT_X'] + extracted_data_raw['coordinate_X']
            )
            extracted_data_raw['wf_y'] = (
                extracted_data_raw['DieY'] * extracted_data_raw['STEP_PITCH_Y'] +
                extracted_data_raw['MAP_SHIFT_Y'] + extracted_data_raw['coordinate_Y']
            )

            ##### context 정보추가 #####
            extracted_data_raw['P_EQPID'] = rawdata_file.iloc[1, 13]
            extracted_data_raw['P_TIME'] = rawdata_file.iloc[2, 13]
            extracted_data_raw['M_TIME'] = rawdata_file.iloc[4, 13]
            extracted_data_raw['Photo_PPID'] = rawdata_file.iloc[11, 13]
            extracted_data_raw['Base_EQP1'] = rawdata_file.iloc[12, 13]
            extracted_data_raw['ChuckID'] = rawdata_file.iloc[15, 13]
            extracted_data_raw['ReticleID'] = rawdata_file.iloc[16, 13]
            
            

            ######## 'Unique_ID'라는 새로운 컬럼 추가   ########  
            ## M_TIME 추가(240922)

            extracted_data_raw['UNIQUE_ID'] = extracted_data_raw.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['M_STEP']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}", axis=1)


            # 'Unique_ID2 추가 (TEST, DieX, DieY 추가. 정렬용)
            extracted_data_raw['UNIQUE_ID2'] = extracted_data_raw.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['M_STEP']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}_{row['TEST']}_{row['DieX']}_{row['DieY']} ", axis=1)
            
                       



            # 컬럼 순서 재조정
            cols_order = [
                'UNIQUE_ID', 'UNIQUE_ID2',
                'STEPSEQ', 'LOT_ID', 'Wafer', 
                'P_EQPID', 'Photo_PPID', 'P_TIME', 'M_STEP', 'M_TIME', 'ChuckID', 'ReticleID', 'Base_EQP1', 
                'TEST', 'DieX', 'DieY',
                'X_reg', 'Y_reg', 'MRC_X', 'MRC_Y', 
                'STEP_PITCH_X', 'STEP_PITCH_Y', 'MAP_SHIFT_X', 'MAP_SHIFT_Y', 'coordinate_X', 'coordinate_Y', 'wf_x', 'wf_y'
            ]
            extracted_data_raw = extracted_data_raw[cols_order]

            # 리스트에 추가
            combined_rawdata_list.append(extracted_data_raw)







            ######################################## Trocs Input 전처리 ##################################################

            trocs_input_file['STEPSEQ'] = stepseq_value_raw
            trocs_input_file['LOT_ID'] = lot_id_value_raw
            trocs_input_file['Wafer'] = wafer_value_raw
            trocs_input_file['P_EQPID'] = p_eqpid__value_raw
            trocs_input_file['Photo_PPID'] = photo_ppid_value_raw
            trocs_input_file['P_TIME'] = p_time_value_raw 
            trocs_input_file['M_TIME'] = m_time_value_raw 
            trocs_input_file['ChuckID'] = chuckid_value_raw
            
            # 'm_step' 신규컬럼 추가
            trocs_input_file['M_STEP'] = m_step_value_raw 

         
            # Trocs Input 데이터프레임에 'Unique_ID'라는 새로운 컬럼 추가
            trocs_input_file['UNIQUE_ID'] = trocs_input_file.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['M_STEP']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}", axis=1)

            # 'Unique_ID2 추가 (DieX(=dCol), DieY(=dRow) 추가. 정렬용)
            trocs_input_file['UNIQUE_ID2'] = trocs_input_file.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['M_STEP']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}_{row['dCol']}_{row['dRow']}", axis=1)



            
            # 순서 재조정
            # 기존 컬럼 뒤로 밀기 - 순서를 맨 앞으로 추가하는 순서로 재배열
            cols_to_insert = ['UNIQUE_ID', 'UNIQUE_ID2', 'STEPSEQ', 'LOT_ID', 'Wafer', 'P_EQPID', 'Photo_PPID', 'P_TIME', 'M_STEP','M_TIME', 'ChuckID']

            # 각 컬럼을 지정된 위치에 삽입 (맨 앞에)
            for i, col in enumerate(cols_to_insert):
                trocs_input_file.insert(i, col, trocs_input_file.pop(col))

            # 리스트에 추가
            combined_trocs_input_list.append(trocs_input_file)





            ######################################## MRC 전처리 ##################################################

            
            # MRC 데이터 추출을 위해 'RawData-1' 시트를 header=None으로 다시 읽음
            rawdata_file_no_header = pd.read_excel(file_path, sheet_name='RawData-1', header=None)

            # MRC 데이터 추출
            mrc_part1 = rawdata_file_no_header.iloc[0:20, 15:17]
            mrc_part2 = rawdata_file_no_header.iloc[22:40, 15:17]
            mrc_part = pd.concat([mrc_part1, mrc_part2], ignore_index=True)

            # 컬럼 이름 설정
            mrc_part.columns = ['K PARA', 'GPM']

            # INDEX 컬럼 추가 (1부터 시작하는 순차적 번호)
            mrc_part['INDEX'] = range(1, len(mrc_part) + 1)


            # 컬럼 추가             
            mrc_part['STEPSEQ'] = stepseq_value_raw
            mrc_part['LOT_ID'] = lot_id_value_raw
            mrc_part['Wafer'] = wafer_value_raw
            mrc_part['P_EQPID'] = p_eqpid__value_raw
            mrc_part['Photo_PPID'] = photo_ppid_value_raw
            mrc_part['P_TIME'] = p_time_value_raw 
            mrc_part['M_TIME'] = m_time_value_raw 
            mrc_part['ChuckID'] = chuckid_value_raw

            # 'm_step' 신규컬럼 추가
            mrc_part['M_STEP'] = m_step_value_raw 




            # mrc_part 데이터프레임에 'Unique_ID'라는 새로운 컬럼 추가
            mrc_part['UNIQUE_ID'] = mrc_part.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['M_STEP']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}", axis=1)


            # 컬럼 순서 재조정
            mrc_cols_order = [
                'UNIQUE_ID',
                'STEPSEQ', 'LOT_ID', 'Wafer', 
                'P_EQPID', 'Photo_PPID', 'P_TIME', 'M_STEP','M_TIME', 'ChuckID',  
                'K PARA', 'GPM', 'INDEX'
            ]
            mrc_part = mrc_part[mrc_cols_order]

            
            # 리스트에 추가
            mrc_data_list.append(mrc_part)

            
            ######################################## PSM 전처리 ##################################################

            psm_input_file['STEPSEQ'] = stepseq_value_raw
            psm_input_file['LOT_ID'] = lot_id_value_raw
            psm_input_file['Wafer'] = wafer_value_raw
            psm_input_file['P_EQPID'] = p_eqpid__value_raw
            psm_input_file['Photo_PPID'] = photo_ppid_value_raw
            psm_input_file['P_TIME'] = p_time_value_raw 
            psm_input_file['M_TIME'] = m_time_value_raw 
            psm_input_file['ChuckID'] = chuckid_value_raw

            # 'm_step' 신규컬럼 추가
            psm_input_file['M_STEP'] = m_step_value_raw 

            # PSM Input 데이터프레임에 'Unique_ID'라는 새로운 컬럼 추가
            psm_input_file['UNIQUE_ID'] = psm_input_file.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['M_STEP']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}", axis=1)

            # 'Unique_ID2 추가 (DieX(=dCol), DieY(=dRow) 추가. 정렬용)
            psm_input_file['UNIQUE_ID2'] = psm_input_file.apply(
                lambda row: f"{row['STEPSEQ']}_{row['P_EQPID']}_{row['Photo_PPID']}_{row['P_TIME']}_{row['M_STEP']}_{row['M_TIME']}_{row['LOT_ID']}_{row['Wafer']}_{row['dCol']}_{row['dRow']}", axis=1)
            
            
            # 순서 재조정
            # 기존 컬럼 뒤로 밀기 - 순서를 맨 앞으로 추가하는 순서로 재배열
            cols_to_insert_psm = ['UNIQUE_ID', 'UNIQUE_ID2', 'STEPSEQ', 'LOT_ID', 'Wafer', 'P_EQPID', 'Photo_PPID', 'P_TIME', 'M_STEP','M_TIME', 'ChuckID']

            # 각 컬럼을 지정된 위치에 삽입 (맨 앞에)
            for i, col in enumerate(cols_to_insert_psm):
                psm_input_file.insert(i, col, psm_input_file.pop(col))

            # 리스트에 추가
            combined_psm_input_list.append(psm_input_file)










    # 리스트를 데이터프레임으로 병합
    combined_rawdata = pd.concat(combined_rawdata_list, ignore_index=True)
    combined_trocs_input = pd.concat(combined_trocs_input_list, ignore_index=True)
    mrc_data = pd.concat(mrc_data_list, ignore_index=True)
    combined_psm_input = pd.concat(combined_psm_input_list, ignore_index=True)

    # 병합 후, UNIQUE_ID기준으로 오름차순 정렬 (내림차순하고 싶으면 ascending=False 옵션 추가하면 됨.) 
    combined_rawdata = combined_rawdata.sort_values(by=['UNIQUE_ID', 'TEST', 'DieX', 'DieY'])    
    combined_trocs_input = combined_trocs_input.sort_values(by=['UNIQUE_ID', 'dCol', 'dRow'])
    mrc_data = mrc_data.sort_values(by=['UNIQUE_ID', 'INDEX'])
    combined_psm_input = combined_psm_input.sort_values(by=['UNIQUE_ID', 'dCol', 'dRow'])


    # 최종 데이터를 엑셀 파일로 저장
    with pd.ExcelWriter('output.xlsx') as writer:
        combined_rawdata.to_excel(writer, sheet_name='RawData-1', index=False)
        combined_trocs_input.to_excel(writer, sheet_name='Trocs Input', index=False)
        mrc_data.to_excel(writer, sheet_name='MRC', index=False)
        combined_psm_input.to_excel(writer, sheet_name='PerShotMRC', index=False)







################################### nau 파일 처리 및 데이터 저장 #####################################################################

print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'nau파일 전처리 시작.')
process_nau_files(folder_path, columns_to_extract)
print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'nau파일 전처리 완료.')






B3N049.1_VH075040_VH075030_20240530171740_11.nau 파일 읽기 성공
B3N049.1_VH075040_VH075030_20240530171740_11.nau 파일에 새로운 시트 추가 완료
PDS211.1_WF075040_WF075030_20240716235504_11.nau 파일 읽기 성공
PDS211.1_WF075040_WF075030_20240716235504_11.nau 파일에 새로운 시트 추가 완료
PRQ328.1_WC046040_WC046030_20240430002132_11.nau 파일 읽기 성공
PRQ328.1_WC046040_WC046030_20240430002132_11.nau 파일에 새로운 시트 추가 완료
2024-10-03 20:41:56 nau파일 전처리 시작.
2024-10-03 20:42:01 nau파일 전처리 완료.
